In [1]:
import sys
import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Transform DBLP2.csv and ACM.csv to lower and
# Get id_to_index
DBLP2=pd.read_csv('csv_files/DBLP2.csv')
DBLP2_lower=open('csv_files/DBLP2_lower.csv','w')
DBLP2_lower.write('"id","title","authors","venue","year"\n')
DBLP2_id_to_index={}
for index, row in DBLP2.iterrows():
    id,title,authors,venue,year=row['id'],row['title'],row['authors'],row['venue'],row['year']
    DBLP2_id_to_index[id]=index
    DBLP2_lower.write('"{}","{}","{}","{}",{}\n'.format(id,title.lower().strip(),authors.lower().strip(),venue.lower().strip(),year))
DBLP2_lower.close()

ACM=pd.read_csv('csv_files/ACM.csv')
ACM_lower=open('csv_files/ACM_lower.csv','w')
ACM_lower.write('"id","title","authors","venue","year"\n')
ACM_id_to_index={}
for index, row in ACM.iterrows():
    id,title,authors,venue,year=row['id'],row['title'],row['authors'],row['venue'],row['year']
    if pd.isnull(authors):
        authors=''
    ACM_id_to_index[id]=index
    ACM_lower.write('{},"{}","{}","{}",{}\n'.format(id,title.lower().strip(),authors.lower().strip(),venue.lower().strip(),year))
ACM_lower.close()

In [2]:
path_A='csv_files/DBLP2_lower.csv'
path_B='csv_files/ACM_lower.csv'

In [3]:
A=em.read_csv_metadata(path_A,key='id')
B=em.read_csv_metadata(path_B,key='id')
em.get_key(A), em.get_key(B)

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


('id', 'id')

In [4]:
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

Number of tuples in A: 2616
Number of tuples in B: 2294
Number of tuples in A X B (i.e the cartesian product): 6001104


In [5]:
A.head()

,id,title,authors,venue,year
0,journals/sigmod/Mackay99,semantic integration of environmental models for application to global information systems and d...,d. scott mackay,sigmod record,1999
1,conf/vldb/PoosalaI96,estimation of query-result distribution and its application in parallel-join load balancing,"viswanath poosala, yannis e. ioannidis",vldb,1996
2,conf/vldb/PalpanasSCP02,incremental maintenance for non-distributive aggregate functions,"themistoklis palpanas, richard sidle, hamid pirahesh, roberta cochrane",vldb,2002
3,conf/vldb/GardarinGT96,cost-based selection of path expression processing algorithms in object-oriented databases,"zhao-hui tang, georges gardarin, jean-robert gruser",vldb,1996
4,conf/vldb/HoelS95,benchmarking spatial join operations with spatial output,"erik g. hoel, hanan samet",vldb,1995


In [6]:
B.head()

,id,title,authors,venue,year
0,304586,the wasa2 object-oriented workflow management system,"gottfried vossen, mathias weske",international conference on management of data,1999
1,304587,a user-centered interface for querying distributed multimedia databases,"isabel f. cruz, kimberly m. james",international conference on management of data,1999
2,304589,"world wide database-integrating the web, corba and databases","athman bouguettaya, boualem benatallah, lily hendra, james beard, kevin smith, mourad quzzani",international conference on management of data,1999
3,304590,xml-based information mediation with mix,"chaitan baru, amarnath gupta, bertram lud&#228;scher, richard marciano, yannis papakonstantinou,...",international conference on management of data,1999
4,304582,the ccube constraint object-oriented database system,"alexander brodsky, victor e. segal, jia chen, paval a. exarkhopoulo",international conference on management of data,1999


In [7]:
# Blocking

# Overlap blocking over input tables on title: the titles of mathched entities should have at least 3 same words
ob=em.OverlapBlocker()
C1=ob.block_tables(A,B,'title','title',word_level=True,overlap_size=1,allow_missing=True,
                 l_output_attrs=['title','authors','venue','year'],
                 r_output_attrs=['title','authors','venue','year'],
                 show_progress=True)
print(len(C1))

# # Overlap blocking over input tables on authors: the authors of matched entities should have at least 1 same word
# C2=ob.block_candset(C1,'authors','authors',word_level=True,overlap_size=1,allow_missing=True,show_progress=True)
# print(len(C2))

# Attribute equivalence blocking on year: the year of matched entities should be the same
ab = em.AttrEquivalenceBlocker()
C2=ab.block_candset(C1,l_block_attr='year',r_block_attr='year',show_progress=True)
print(len(C2))

/Users/xuedi/anaconda3/envs/deepbrowse/lib/python3.6/site-packages/py_stringsimjoin/utils/converter.py:99: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  if col_type == pd.np.object:
/Users/xuedi/anaconda3/envs/deepbrowse/lib/python3.6/site-packages/py_entitymatching/blocker/overlap_blocker.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df[l_dummy_overlap_attr] = l_df[l_overlap_attr]
/Users/xuedi/anaconda3/envs/deepbrowse/lib/python3.6/site-packages/py_entitymatching/blocker/overlap_blocker.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

Finding pairs with missing value...
1953933


0% [##############################] 100% | ETA: 00:00:00

200630



Total time elapsed: 00:00:07


In [8]:
C2.columns

Index(['_id', 'ltable_id', 'rtable_id', 'ltable_title', 'ltable_authors',
       'ltable_venue', 'ltable_year', 'rtable_title', 'rtable_authors',
       'rtable_venue', 'rtable_year'],
      dtype='object')

In [9]:
dblp_match_dic={}
acm_match_dic={}
match=pd.read_csv('csv_files/DBLP-ACM_perfectMapping.csv')
length=len(match)
for i in range(length):
    idDBLP=match['idDBLP'][i]
    idACM=match['idACM'][i]
    if idDBLP in dblp_match_dic:
        dblp_match_dic[idDBLP].append(idACM)
    else:
        dblp_match_dic[idDBLP]=[idACM]
    if idACM in acm_match_dic:
        acm_match_dic[idACM].append(idDBLP)
    else:
        acm_match_dic[idACM]=[idDBLP]
for idDBLP in dblp_match_dic:
    if len(dblp_match_dic[idDBLP])>1:
        print(idDBLP,dblp_match_dic[idDBLP])
for idACM in acm_match_dic:
    if len(acm_match_dic[idACM])>1:
        print(idACM,acm_match_dic[idACM])

len(dblp_match_dic),len(acm_match_dic)

(2224, 2224)

In [10]:
import py_entitymatching as em


def jac_qgm_3_qgm_3(seq1, seq2):
    qgm1 = []
    qgm2 = []
    seq1 = '##' + seq1 + '$$'
    seq2 = '##' + seq2 + '$$'
    for i in range(len(seq1) - 2):
        qgm1.append(seq1[i:i + 3])
    for i in range(len(seq2) - 2):
        qgm2.append(seq2[i:i + 3])
    qgm1 = set(qgm1)
    qgm2 = set(qgm2)
    return len(qgm1 & qgm2) / len(qgm1 | qgm2)

def cos_dlm_dc0_dlm_dc0(seq1, seq2):
    set1=set(seq1.split(' '))
    set2=set(seq2.split(' '))
    return float(len(set1 & set2)) / (math.sqrt(float(len(set1))) * math.sqrt(float(len(set2))))

def mel(seq1,seq2):
    return em.monge_elkan(seq1,seq2)

def lev_sim(seq1,seq2):
    return em.lev_sim(seq1,seq2)
    
def exm(seq1,seq2):
    return seq1 == seq2

def anm(seq1,seq2):
    return em.abs_norm(seq1,seq2)


hhh_blocking=open('csv_files/hhh_blocking.csv','w')
hhh_blocking.write('_id,idDBLP,idACM,title_title_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,venue_venue_jac_qgm_3_qgm_3,year_year_lev_sim,gold\n')


_id=0
for index, row in C2.iterrows():
    ltable_id=row['ltable_id']
    rtable_id=row['rtable_id']
    
    ltable_title=row['ltable_title']
    rtable_title=row['rtable_title']
    title_title_jac_qgm_3_qgm_3=jac_qgm_3_qgm_3(ltable_title,rtable_title)
    
    ltable_authors=row['ltable_authors']
    rtable_authors=row['rtable_authors']
    if pd.isnull(ltable_authors):
        ltable_authors=''
    if pd.isnull(rtable_authors):
        rtable_authors=''
    authors_authors_jac_qgm_3_qgm_3=jac_qgm_3_qgm_3(ltable_authors,rtable_authors)
    
    ltable_venue=row['ltable_venue']
    rtable_venue=row['rtable_venue']
    if pd.isnull(ltable_venue):
        ltable_venue=''
    if pd.isnull(rtable_venue):
        rtable_venue=''
    venue_venue_jac_qgm_3_qgm_3=jac_qgm_3_qgm_3(ltable_venue,rtable_venue)
    
    ltable_year=row['ltable_year']
    rtable_year=row['rtable_year']
    if pd.isnull(ltable_year):
        ltable_year=''
    if pd.isnull(rtable_year):
        rtable_year=''
    year_year_lev_sim=lev_sim(ltable_year,rtable_year)
    
    
    if ltable_id in dblp_match_dic:
        if rtable_id in dblp_match_dic[ltable_id]:
            gold=1
        else:
            gold=0
    else:
        gold=0
    
    hhh_blocking.write('{},{},{},{},{},{},{},{}\n'.format(_id,ltable_id,rtable_id,title_title_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,venue_venue_jac_qgm_3_qgm_3,year_year_lev_sim,gold))
    
    _id+=1
    
    
hhh_blocking.close()


In [13]:
def jac_qgm_3_qgm_3(seq1, seq2):
    qgm1 = []
    qgm2 = []
    seq1 = '##' + seq1 + '$$'
    seq2 = '##' + seq2 + '$$'
    for i in range(len(seq1) - 2):
        qgm1.append(seq1[i:i + 3])
    for i in range(len(seq2) - 2):
        qgm2.append(seq2[i:i + 3])
    qgm1 = set(qgm1)
    qgm2 = set(qgm2)
    return len(qgm1 & qgm2) / len(qgm1 | qgm2)
jac_qgm_3_qgm_3('vldb','very large data bases')

0.03571428571428571

In [2]:
hhh=pd.read_csv('csv_files/hhh_blocking.csv')

In [3]:
m_data=hhh[hhh.gold==1]
u_data=hhh[hhh.gold==0]

In [7]:
import numpy as np
np.unique(m_data['year_year_lev_sim'])

array([1.])

In [14]:
m_data.head()

,_id,idDBLP,idACM,title_title_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,venue_venue_jac_qgm_3_qgm_3,year_year_lev_sim,gold
50,50,conf/sigmod/VossenW99,304586,1.000000,0.609756,0.155172,1.0,1
112,112,conf/sigmod/CruzJ99,304587,1.000000,0.627907,0.155172,1.0,1
173,173,conf/sigmod/BouguettayaBH99,304589,0.793651,0.792079,0.155172,1.0,1
262,262,conf/sigmod/BaruGLMPVC99,304590,1.000000,0.750000,0.155172,1.0,1
311,311,conf/sigmod/BrodskySCE99,304582,1.000000,0.813333,0.155172,1.0,1


In [15]:
# Explore m_data

# Output the matched paires whose title similarity is not 1
min_title_sim=1
for index, row in m_data.iterrows():
    if row['title_title_jac_qgm_3_qgm_3']<1:
        if row['title_title_jac_qgm_3_qgm_3']<min_title_sim:
            min_title_sim=row['title_title_jac_qgm_3_qgm_3']
            print('idDBLP:{}, idACM:{}, title_title_jac_qgm_3_qgm_3:{}'.format(row['idDBLP'], row['idACM'], row['title_title_jac_qgm_3_qgm_3']))
print('min_title_sim:{}\n'.format(min_title_sim))

# Output the distinct venue sim value of matched pairs
min_venue_sim=1
venueSim_to_venueValue={}
for index, row in m_data.iterrows():
    venueSim=row['venue_venue_jac_qgm_3_qgm_3']
    venueValue=((A.iloc[DBLP2_id_to_index[row['idDBLP']]])['venue'],(B.iloc[ACM_id_to_index[row['idACM']]])['venue'])
    if venueSim not in venueSim_to_venueValue:
        venueSim_to_venueValue[venueSim]=[venueValue]
        if venueSim<min_venue_sim:
            min_venue_sim=venueSim
    else:
        if venueValue not in venueSim_to_venueValue[venueSim]:
            venueSim_to_venueValue[venueSim].append(venueValue)
print(venueSim_to_venueValue)
print('min_venue_sim:{}\n'.format(min_venue_sim))

# Output the distinct year value of matched pairs
min_year_sim=1
yearSim_to_yearValue={}
for index, row in m_data.iterrows():
    yearSim=row['year_year_lev_sim']
    yearValue=((A.iloc[DBLP2_id_to_index[row['idDBLP']]])['year'],(B.iloc[ACM_id_to_index[row['idACM']]])['year'])
    if yearSim not in yearSim_to_yearValue:
        yearSim_to_yearValue[yearSim]=[yearValue]
        if yearSim<min_year_sim:
            min_year_sim=yearSim
    else:
        if yearValue not in yearSim_to_yearValue[yearSim]:
            yearSim_to_yearValue[yearSim].append(yearValue)
print(yearSim_to_yearValue)
print('min_year_sim:{}\n'.format(min_year_sim))


idDBLP:conf/sigmod/BouguettayaBH99, idACM:304589, title_title_jac_qgm_3_qgm_3:0.7936507936507936
idDBLP:conf/sigmod/BonnetBCCCHKMMOSS99, idACM:304583, title_title_jac_qgm_3_qgm_3:0.75
idDBLP:conf/sigmod/Ojjeh99, idACM:304245, title_title_jac_qgm_3_qgm_3:0.3908045977011495
idDBLP:journals/sigmod/Dogac02, idACM:507340, title_title_jac_qgm_3_qgm_3:0.3648648648648649
idDBLP:journals/sigmod/Winslett02b, idACM:601871, title_title_jac_qgm_3_qgm_3:0.2262773722627737
idDBLP:journals/sigmod/Labrinidis01, idACM:604283, title_title_jac_qgm_3_qgm_3:0.2222222222222222
idDBLP:journals/sigmod/Winslett03, idACM:640999, title_title_jac_qgm_3_qgm_3:0.15
min_title_sim:0.15

{0.15517241379310345: [('sigmod conference', 'international conference on management of data')], 0.6190476190476191: [('sigmod record', 'acm sigmod record')], 0.4117647058823529: [('acm trans. database syst.', 'acm transactions on database systems (tods)')], 0.057142857142857134: [('vldb j.', 'the vldb journal &mdash; the international

In [16]:
u_data.head()

,_id,idDBLP,idACM,title_title_jac_qgm_3_qgm_3,authors_authors_jac_qgm_3_qgm_3,venue_venue_jac_qgm_3_qgm_3,year_year_lev_sim,gold
0,0,conf/sigmod/BraumandlKK99,304586,0.059524,0.012048,0.155172,1.0,0
1,1,conf/vldb/Graefe99,304586,0.041237,0.068182,0.000000,1.0,0
2,2,journals/vldb/HarderST99,304586,0.030303,0.000000,0.000000,1.0,0
3,3,conf/sigmod/IvesFFLW99,304586,0.058824,0.068627,0.155172,1.0,0
4,4,journals/sigmod/EisenbergM99a,304586,0.027273,0.016129,0.000000,1.0,0


In [18]:
# Explore u_data

# Output the max title sim value of unmatched pairs
max_title_sim=0
for index, row in u_data.iterrows():
    if row['title_title_jac_qgm_3_qgm_3']>max_title_sim:
        max_title_sim=row['title_title_jac_qgm_3_qgm_3']
        print('idDBLP:{}, idACM:{}, title_title_jac_qgm_3_qgm_3:{}'.format(row['idDBLP'], row['idACM'], row['title_title_jac_qgm_3_qgm_3']))
print('max_title_sim:{}\n'.format(max_title_sim))

# Output the max authors sim value of unmatched pairs
max_authors_sim=0
for index, row in u_data.iterrows():
    if row['authors_authors_jac_qgm_3_qgm_3']>max_authors_sim:
        max_authors_sim=row['authors_authors_jac_qgm_3_qgm_3']
        print('idDBLP:{}, idACM:{}, authors_authors_jac_qgm_3_qgm_3:{}'.format(row['idDBLP'], row['idACM'], row['authors_authors_jac_qgm_3_qgm_3']))
print('max_authors_sim:{}\n'.format(max_authors_sim))

# Output the max venue sim value of unmatched pairs
max_venue_sim=0
for index, row in u_data.iterrows():
    if row['venue_venue_jac_qgm_3_qgm_3']>max_venue_sim:
        max_venue_sim=row['venue_venue_jac_qgm_3_qgm_3']
        print('idDBLP:{}, idACM:{}, venue_venue_jac_qgm_3_qgm_3:{}'.format(row['idDBLP'], row['idACM'], row['venue_venue_jac_qgm_3_qgm_3']))
print('max_venue_sim:{}\n'.format(max_venue_sim))

for index, row in u_data.iterrows():
    if row['title_title_jac_qgm_3_qgm_3']>0.8 and \
        row['authors_authors_jac_qgm_3_qgm_3']==1:
        print('idDBLP:{}, idACM:{}'.format(row['idDBLP'], row['idACM']))

idDBLP:conf/sigmod/BraumandlKK99, idACM:304586, title_title_jac_qgm_3_qgm_3:0.05952380952380953
idDBLP:journals/sigmod/PapianiWDN99, idACM:304586, title_title_jac_qgm_3_qgm_3:0.060869565217391314
idDBLP:conf/vldb/Norvag99, idACM:304586, title_title_jac_qgm_3_qgm_3:0.22123893805309736
idDBLP:journals/tods/CasatiCPP99, idACM:304586, title_title_jac_qgm_3_qgm_3:0.2551020408163265
idDBLP:conf/sigmod/BrodskySCE99, idACM:304586, title_title_jac_qgm_3_qgm_3:0.34177215189873417
idDBLP:conf/sigmod/AcharyaGPR99a, idACM:304581, title_title_jac_qgm_3_qgm_3:0.4153846153846154
idDBLP:journals/sigmod/LiefkeS00, idACM:335405, title_title_jac_qgm_3_qgm_3:0.43859649122807015
idDBLP:conf/sigmod/Ellison95, idACM:277955, title_title_jac_qgm_3_qgm_3:1.0
max_title_sim:1.0

idDBLP:conf/sigmod/BraumandlKK99, idACM:304586, authors_authors_jac_qgm_3_qgm_3:0.012048192771084338
idDBLP:conf/vldb/Graefe99, idACM:304586, authors_authors_jac_qgm_3_qgm_3:0.06818181818181818
idDBLP:conf/sigmod/IvesFFLW99, idACM:304586, 

In [46]:
# clean golden_records_manually.csv

import pandas as pd

golden_records_manually=pd.read_csv('csv_files/golden_records_manually.csv')
golden_records_cleaned=open('csv_files/golden_records_cleaned.csv','w')
golden_records_cleaned.write('cluster_id,Title,Authors,Venue,Year,Citations,Source\n')

for index, row in golden_records_manually.iterrows():
    cluster_id=row['cluster_id']
    title=row['Title']
    authors=row['Authors']
    venue=row['Venue']
    year=row['Year']
    citations=row['Citations']
    source=row['Source']
    authors=authors.replace(',',', ')
    authors=authors.replace(',   ',', ')
    authors=authors.replace(',  ',', ')
    authors=authors.lower()
    golden_records_cleaned.write('{},"{}","{}",{},{},{},{}\n'.format(cluster_id,title,authors,venue,year,citations,source))
    
golden_records_cleaned.close()    

In [9]:
import pandas as pd
import numpy as np

dblp=pd.read_csv('csv_files/DBLP2_lower.csv')
acm=pd.read_csv('csv_files/ACM_lower.csv')
print(list(np.unique(dblp['venue'])))
print(list(np.unique(acm['venue'])))

['acm trans. database syst.', 'sigmod conference', 'sigmod record', 'vldb', 'vldb j.']
['acm sigmod record', 'acm transactions on database systems (tods)', 'international conference on management of data', 'the vldb journal &mdash; the international journal on very large data bases', 'very large data bases']
